- A guide to make asynchronous calls to OpenAI API with rate limiting.
- Backoff is a python library that provides decorators that can used to wrap a function and retry until a specified condition is met.

### Asyncio

- Asyncio is a Python library that provides asynchronous I/O.
    - 协程（coroutine）：使用async def定义的函数
        - 可以在执行过程中暂停和恢复
        - 必须使用await来调用
    - await 关键字
        - 用于暂停协程执行，等待异步操作完成
        - 只能在async函数内使用
    - asyncio.run
        - 执行 async def 定义的事件循环（event loop）
    - asyncio.gather()同时处理多个异步任务
- It relieves you of the burden of worrying about **locks and threads** when writing concurrent programs.
- The foundation of asyncio is the idea of **coroutines**, or functions that can be interrupted and then resumed.

- CPU密集型任务不适合异步编程的原因
    - 异步编程的优势在于I/O等待时的切换

```python
# I/O密集型任务 - 适合异步
async def io_task():
    await asyncio.sleep(1)  # 在等待I/O时，可以切换到其他任务
    return "完成"

# CPU密集型任务 - 不适合异步
async def cpu_task():
    result = 0
    for i in range(1000000000):  # CPU计算，无法切换
        result += i
    return result
```

### semaphore

- asyncio.Semaphore：控制并发；
    - 用于限制同时运行的协程数量，从而防止资源过载或满足某些并发限制需求。
- asyncio.as_completed：获取实时结果；
    - asyncio.as_completed 会在每个任务完成时生成一个迭代器，使您能够立即处理完成的任务，而不必等待所有任务结束。